In [3]:
import os
import time 
import numpy as np
from __future__ import print_function

import tensorflow as tf

C:\Users\ericmlyang\AppData\Local\Continuum\anaconda3\envs\tf_cpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
# return: batch_generator
#g = batch_generator(arr, FLAGS.num_seqs, FLAGS.num_steps)

def batch_generator(input_len=40, output_len=40, batch_size=1, total=16000, seq_len=1):
    runs = int(total/seq_len/batch_size)
    
    input_data = []
    target_data = []
    
    for run in range(runs):
        x = np.zeros(shape=(batch_size, seq_len, input_len))
        y = np.zeros(shape=(batch_size, seq_len, output_len))
        for seq in range(seq_len):
            for batch in range(batch_size):
                sequen = np.random.randint(2, size=input_len)
                x[batch, seq, :] = sequen
                y[batch, seq, :] = np.abs(sequen - 1)
                
        input_data.append(x)
        target_data.append(y)
                
    return input_data, target_data


In [16]:
class N2N_LSTM:
    def __init__(self,
                 seq_max_len=1,  
                 input_len=40,
                 output_len=40,
                 batch_size=1,
                 lstm_size=[40, 40, 40],
                 num_layers=3,
                 learning_rate=0.001,
                 grad_clip=2,
                 keep_prob=1.,
                 ):
            
        self.seq_max_len = seq_max_len
        self.input_len = input_len
        self.output_len = output_len
        self.batch_size = batch_size
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.learning_rate = learning_rate
        self.grad_clip = grad_clip
        self.train_keep_prob = keep_prob
        
        tf.reset_default_graph() #Clears the default graph stack and resets the global default graph
        self.build_inputs()
        self.build_lstm()
        self.build_loss()
        self.build_optimizer()
        self.saver = tf.train.Saver() #Saves and restores variables.
 

    
    def build_inputs(self):
        with tf.name_scope('inputs'):
            self.inputs = tf.placeholder(tf.float32, 
                                         shape=(self.batch_size, self.seq_max_len, self.input_len),
                                         name='inputs')
            self.targets = tf.placeholder(tf.float32,
                                          shape=(self.batch_size, self.seq_max_len, self.output_len),
                                          name='targets')
            self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
            
            
            
    def build_lstm(self):
        def get_a_cell(lstm_size, keep_prop):
            lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
            drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=self.train_keep_prob)
            return drop
        
        # tf.orthogonal_initializer()
        # tf.truncated_normal_initializer()
        with tf.variable_scope('lstm', initializer=tf.orthogonal_initializer()):
            cell = tf.nn.rnn_cell.MultiRNNCell(
                         [get_a_cell(size, self.keep_prob) for size in self.lstm_size]
                                              )
            self.initial_state = cell.zero_state(self.batch_size, tf.float32)
            
            # 透過dynamic_rnn對cell展開時間維度
            self.lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cell, 
                                                                    self.inputs, 
                                                                    initial_state=self.initial_state)
            #print(self.lstm_outputs)
            
            
            '''
            # 透過lstm_outputs得到機率
            seq_output = tf.concat(self.lstm_outputs, 1)
            x = tf.reshape(seq_output, [-1, self.lstm_size[-1]])

            with tf.variable_scope('sigmoid'):
                sigmoid_w = tf.Variable(tf.truncated_normal([self.lstm_size[-1],self.output_len], stddev=0.1))
                sigmoid_b = tf.Variable(tf.zeros(self.output_len))

            self.logits = tf.matmul(x, sigmoid_w) + sigmoid_b
            '''
            self.proba_prediction = tf.tanh(self.lstm_outputs, name='predictions')

            
            
            
    def build_loss(self):
        with tf.name_scope('loss'):
            self.y_reshaped = tf.reshape(self.targets, self.lstm_outputs.get_shape())
            loss =tf.losses.mean_squared_error(predictions=self.lstm_outputs, labels=self.y_reshaped)
            #loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, labels=self.y_reshaped)
            self.loss = tf.reduce_mean(loss)
            
            
            
            
    def build_optimizer(self):
        # 使用clipping gradients
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), self.grad_clip)
        train_op = tf.train.AdamOptimizer(self.learning_rate)
        self.optimizer = train_op.apply_gradients(zip(grads, tvars))


            

    def train(self, x, y, iters=10, save_path='./models', save_every_n=200, log_every_n=200):
        self.session = tf.Session()
        with self.session as sess:
            sess.run(tf.global_variables_initializer())
            # Train network
            
            new_state = sess.run(self.initial_state)
            for ite in range(iters):
                step = 0
                print('iters',ite)
                for i in range(len(x)):
                    step += 1
                    start = time.time()
                    feed = {self.inputs: x[i], 
                            self.targets: y[i],
                            self.keep_prob: self.train_keep_prob,
                            self.initial_state: new_state}
                    batch_loss, new_state, _, pred, output, target = sess.run([self.loss,
                                                         self.final_state,
                                                         self.optimizer,
                                                         self.proba_prediction,
                                                         self.lstm_outputs,
                                                         self.y_reshaped],
                                                        feed_dict=feed)
                    # print result
                    #self.print_result(x[i], y[i])
                    end = time.time()

                    # control the print lines
                    if step % log_every_n == 0:
                        print("=======================================================\n")
                        print('step: {} in iter: {}/{}... '.format(step, ite+1, iters),
                              'loss: {:.4f}... '.format(batch_loss),
                              '{:.4f} sec/batch'.format((end - start)))
                        print("FIRST OUTPUT: \n",output)
                        print("Target: \n", target)
                        print("PRED: \n",pred)
                        
                    if (step % save_every_n == 0):
                        self.saver.save(sess, os.path.join(save_path, 'model'), global_step=step)      

In [17]:
model_path = './models'
if os.path.exists(model_path) is False:
    os.makedirs(model_path)
input_data, target_data = batch_generator()
model = N2N_LSTM()
model.train(input_data, target_data,
            iters=5,
            save_path=model_path,
            save_every_n=200,
            log_every_n =200
            )

iters 0

step: 200 in iter: 1/5...  loss: 0.2563...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[0.5461823  0.47950023 0.40281984 0.47676155 0.513213   0.5071213
   0.5769264  0.5480253  0.4116828  0.42490074 0.5042709  0.5286763
   0.4810581  0.52873737 0.4756594  0.54656464 0.4549535  0.54187644
   0.5185916  0.44750422 0.42452687 0.39915255 0.60362965 0.51934725
   0.45795828 0.47866696 0.584155   0.46359164 0.48587316 0.4447369
   0.47997257 0.53776115 0.6367721  0.51885587 0.4585336  0.49194732
   0.5578801  0.47175914 0.5553582  0.50967443]]]
Target: 
 [[[0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1.
   1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]]
PRED: 
 [[[0.49765342 0.44584328 0.38235912 0.44364628 0.47244483 0.4676992
   0.5204279  0.49903876 0.38990062 0.40105096 0.46546936 0.48436856
   0.44709054 0.48441535 0.4427606  0.49794102 0.42596227 0.49440706
   0.47661227 0.41984528 0.40073723 0.37922361 0.5396273  0.47719607
   0.42841867 0.4451754  0.5256


step: 1600 in iter: 1/5...  loss: 0.1925...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[0.71856505 0.54033256 0.49578744 0.5300746  0.33239684 0.8261012
   0.51647574 0.5162762  0.58554065 0.52985024 0.45704073 0.6084615
   0.5629154  0.41632202 0.545322   0.62553996 0.70556146 0.39561173
   0.35279977 0.3825644  0.3970315  0.2128435  0.5006428  0.40718514
   0.31335527 0.5089545  0.3054983  0.80499923 0.30362025 0.5926765
   0.58916384 0.57776684 0.54687333 0.57424235 0.609112   0.28934798
   0.3501335  0.2525581  0.3041604  0.35556594]]]
Target: 
 [[[1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0.]]]
PRED: 
 [[[0.6160197  0.49323964 0.45879775 0.4854381  0.32067278 0.678377
   0.47497538 0.47482088 0.5266808  0.4852666  0.4276693  0.5430432
   0.5101373  0.3938274  0.4970059  0.55497354 0.6078861  0.37618798
   0.33885616 0.36493242 0.37740612 0.20968656 0.46262252 0.38607997
   0.30348644 0.46913022 0.29633617 0.66


step: 3000 in iter: 1/5...  loss: 0.1673...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[0.16132204 0.35874096 0.6578558  0.8369329  0.4941683  0.6659238
   0.1298717  0.20213574 0.7164435  0.2537306  0.33335316 0.507579
   0.3009623  0.5991778  0.4249022  0.5484989  0.3180089  0.25721452
   0.65820074 0.81452584 0.2761844  0.4832827  0.44086656 0.62773836
   0.54203254 0.7059528  0.67752594 0.61528826 0.50209224 0.8621979
   0.40894973 0.59043735 0.14455833 0.8572537  0.48595992 0.4327622
   0.35924432 0.6920328  0.7657407  0.31112346]]]
Target: 
 [[[0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1.
   1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0.]]]
PRED: 
 [[[0.159937   0.34410453 0.57693464 0.6841811  0.45751852 0.5822922
   0.12914641 0.199427   0.6147015  0.24842225 0.3215305  0.46805677
   0.29219297 0.5364642  0.4010522  0.49939433 0.30770543 0.2516883
   0.57716477 0.67207944 0.26937    0.44886872 0.41436246 0.556493
   0.49452502 0.60813266 0.58990866 0.54783


step: 4400 in iter: 1/5...  loss: 0.1622...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[0.80800194 0.8604092  0.6472888  0.19096968 0.5138293  0.7463849
   0.8041475  0.3155336  0.51138747 0.54798174 0.16680962 0.6143872
   0.25353146 0.14606671 0.48239425 0.67304176 0.77314854 0.7378813
   0.06092887 0.4239035  0.38381395 0.4021388  0.83847636 0.37833124
   0.5643001  0.57476264 0.5315919  0.9606672  0.43986934 0.649542
   0.26913667 0.54120916 0.52444965 0.6670321  0.5165677  0.46245205
   0.1340226  0.42690724 0.7668857  0.518213  ]]]
Target: 
 [[[1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1.
   0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1.]]]
PRED: 
 [[[0.6684866  0.69646853 0.569842   0.18868151 0.47292346 0.6329874
   0.666349   0.3054628  0.47102553 0.499006   0.16527945 0.5472081
   0.24823537 0.14503668 0.448159   0.58697724 0.64875656 0.6278634
   0.06085359 0.40021378 0.36601508 0.3817775  0.6850013  0.36125734
   0.51116097 0.51884836 0.48659697 0.7445


step: 5800 in iter: 1/5...  loss: 0.1677...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[0.911343   0.75039035 0.30678576 0.19708695 0.85075617 0.50212985
   0.22087684 0.8734365  0.4110696  0.34915996 0.40168414 0.14330444
   0.44598335 0.6929058  0.8772803  0.18329468 0.701621   0.20954819
   0.6738642  0.8666081  0.21770939 0.18133333 0.4666505  0.19486977
   0.738336   0.6931703  0.23943073 0.6587006  0.5545974  0.93536824
   0.7937073  0.606214   0.4681551  0.65511096 0.35753193 0.05119589
   0.05852632 0.67140687 0.48623347 0.58637697]]]
Target: 
 [[[0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
   0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1.]]]
PRED: 
 [[[0.7217763  0.6353818  0.29751015 0.19457415 0.69146436 0.4637905
   0.21735364 0.70311594 0.38938048 0.3356303  0.38138902 0.14233144
   0.41859174 0.5998455  0.7050543  0.18126918 0.60539573 0.20653398
   0.587516   0.6996468  0.21433373 0.17937161 0.43548897 0.19243997
   0.6281388  0.60001487 0.23495796

...


step: 6800 in iter: 5/5...  loss: 0.0186...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[ 0.22215076  0.8274201   0.00693482  0.03196272  0.15012808
    0.03978856  0.97375983  0.9980738   0.21866973  0.05806706
    0.04070885  0.03520806  0.9811832   0.6272324   0.71985716
    0.07312129  0.00822004  0.01187732  0.7260229   0.00554286
    0.99537235  0.969144    0.01805533  0.7626441   0.01063817
    0.9954087   0.12213481 -0.00919544  0.8722156   0.987394
    0.99417514  0.9854525   0.12643385  0.74645346  0.23340619
    0.9874443   0.10184665  0.9900581   0.03890053  0.22405702]]]
Target: 
 [[[0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0.
   1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0.]]]
PRED: 
 [[[ 0.218567    0.6790883   0.00693471  0.03195184  0.14901026
    0.03976757  0.7503519   0.7607841   0.21524975  0.05800189
    0.04068637  0.03519351  0.7535777   0.5561436   0.6168209
    0.07299124  0.00821985  0.01187676  0.62062615  0.0055428
    0.75964385  0.7


step: 8000 in iter: 5/5...  loss: 0.0324...  0.0020 sec/batch
FIRST OUTPUT: 
 [[[ 8.1963408e-01  1.3086782e-02  1.8339753e-02  8.6168028e-02
    9.7053951e-01  8.0120105e-01  3.2757543e-02  1.3344161e-03
    9.9536681e-01  9.1640389e-01  4.6806440e-01  7.0933586e-01
    1.8300759e-02 -2.3596475e-02 -4.5049772e-02 -1.0605805e-02
    9.9327612e-01  2.3812205e-01 -2.0116614e-03  8.1592929e-03
    3.0911994e-01  9.4292486e-01  6.9552362e-01  5.3569779e-02
    9.0480286e-01  9.5273668e-01  5.0792538e-02  7.7180564e-01
    7.1511376e-01  4.6717785e-03  8.6109161e-01  6.3895889e-02
    9.3288499e-01  9.2889577e-01  3.4943458e-02  9.9003632e-03
    7.2714692e-01  4.0236048e-02  9.0917492e-01  7.4238435e-04]]]
Target: 
 [[[1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1.
   0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0.]]]
PRED: 
 [[[ 6.7487073e-01  1.3086034e-02  1.8337695e-02  8.5955389e-02
    7.4894136e-01  6.6470772e-01  3.2745827e-02  1.3344151e-03
    7.5964147e-


step: 9200 in iter: 5/5...  loss: 0.0566...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[ 6.68413281e-01  6.52401388e-01  2.67003085e-02  4.40120280e-01
    2.87745651e-02  3.90050620e-01  6.73991859e-01  9.87486482e-01
    1.18893264e-02  9.47027445e-01  1.26538165e-02  1.13202654e-01
    3.69011541e-03  4.38345550e-03  9.89278436e-01  4.39450115e-01
    4.04377319e-02  4.08584654e-01  2.81292978e-06  9.31851149e-01
    8.19001019e-01 -4.61095534e-02  7.38163114e-01  3.90977971e-02
    1.95985660e-02  5.68103790e-03  9.96777952e-01  7.43986428e-01
   -1.93627421e-02  9.98948991e-01  9.62545514e-01  9.62012768e-01
    8.30095589e-01  7.69347370e-01  2.33956575e-02  1.57714844e-01
   -6.79960921e-02  6.90234005e-01  9.00191486e-01  1.89854868e-03]]]
Target: 
 [[[1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1.
   0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]]]
PRED: 
 [[[ 5.8393520e-01  5.7328439e-01  2.6693964e-02  4.1374412e-01
    2.8766623e-02  3.7140384e-01  5.87


step: 10400 in iter: 5/5...  loss: 0.0318...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[-5.12956008e-02  3.06915701e-03  8.63556683e-01  9.84833658e-01
    1.04713589e-01  9.64332663e-04  5.29702790e-02  5.95047213e-02
    8.51990819e-01  9.74778891e-01  9.96618390e-01  9.68595445e-01
    3.25635910e-01 -4.01987284e-02 -1.34704299e-02  2.68086344e-01
    9.91469085e-01  4.52317685e-01 -2.17888085e-03  2.02891320e-01
    1.28269549e-02 -5.10383248e-02  7.35718429e-01  1.02210594e-02
    9.94088292e-01  4.66306601e-03  9.95486796e-01  7.81275868e-01
    3.40016663e-01  8.46192956e-01  2.90257633e-02  1.05800135e-02
    5.95564693e-02  9.92098033e-01  8.66551340e-01  1.47510543e-01
   -5.25747053e-02  3.78800929e-01  1.02623338e-02  8.86444449e-01]]]
Target: 
 [[[0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
   0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.]]]
PRED: 
 [[[-0.05125065  0.00306915  0.69808567  0.7551508   0.10433252
    0.00096433  0.05292079  0.0594345


step: 11600 in iter: 5/5...  loss: 0.0125...  0.0010 sec/batch
FIRST OUTPUT: 
 [[[ 2.1700077e-03  9.7552466e-01  3.3995166e-01  9.9777228e-01
    1.7169115e-01  6.8226653e-01  3.4444254e-02  9.8142886e-01
    5.8293948e-03  9.9871814e-01  8.0097429e-02  1.6289691e-02
    9.0208876e-01  9.2432064e-01 -1.9909684e-03  9.9461317e-01
    9.4114608e-01  6.0097639e-02 -1.7229872e-02  9.8744094e-01
    4.3520793e-02 -6.3715562e-02  9.3831319e-01  1.9203047e-01
    2.5295755e-03  8.2029356e-04  9.9874616e-01  7.2111353e-02
   -5.6277342e-02  3.0990124e-02  3.9244619e-01  5.7453562e-02
    9.8585039e-01 -1.3554071e-02  3.9573509e-02  7.0500247e-02
    9.9150842e-01  9.9640590e-01  9.7191173e-01  9.6332389e-01]]]
Target: 
 [[[0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1.
   0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1.]]]
PRED: 
 [[[ 0.00217     0.75112206  0.3274342   0.7606571   0.17002377
    0.59299093  0.03443063  0.7536838   0.00582933  0.7610554
    0.07992657 